In [1]:
!gdown  18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 79.5MB/s]


In [ ]:
!unzip scc5966.zip

In [ ]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [ ]:
import pandas as pd
from tqdm import tqdm 

df=pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_train_data[df_train_data.movie_id == movie]
  df.loc[temp.user_id, movie] = temp.rating.to_list()

100%|██████████| 3562/3562 [00:06<00:00, 571.54it/s]


In [ ]:
from scipy.stats import pearsonr
import numpy as np

In [ ]:
cont = 1
dic_sim = {}

for item1 in tqdm(df.columns):
  for item2 in df.columns[cont:]:
    
    x = np.array(df[item1], dtype=float)
    y = np.array(df[item2], dtype=float)

    bad = ~np.logical_or(np.isnan(x), np.isnan(y))

    x = np.compress(bad, x)  
    y = np.compress(bad, y)  

    if len(x) > 1:
      similarity = pearsonr(x,y)[0]
      if similarity is np.nan:
        similarity = 0
    else:
      similarity = 0

    dic_sim[(item1,item2)] = similarity

  cont+=1

  0%|          | 0/3562 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
100%|██████████| 3562/3562 [36:37<00:00,  1.62it/s] 


In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_itens_pearson.p', 'wb') as fp:
    pickle.dump(dic_sim, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_itens_pearson.p', 'rb') as fp:
    data = pickle.load(fp)

In [ ]:
def similarity_itens(similarities, neighbor_item, item):
  
  if (item,neighbor_item) not in similarities.keys():
    similarity = similarities[(neighbor_item,item)]
  else:
    similarity = similarities[(item,neighbor_item)]
    
  return similarity
  
def k_neighbors_nearest(df, similarities, k, user, item):
  k_neighbors_similarity = [-2] * k
  k_neighbors = [-1] * k

  for neighbor_item in df.columns:
    
    ni = neighbor_item
    if item != ni and df.loc[user][ni] is not np.nan:
      
      similarity = similarity_itens(similarities, ni, item)

      for i in range(k):
        if similarity > k_neighbors_similarity[i]:
          aux = k_neighbors_similarity[i]
          k_neighbors_similarity[i] = similarity
          similarity = aux

          aux = k_neighbors[i]
          k_neighbors[i] = ni
          ni = aux

  return k_neighbors

def pred(df, similarities, k, user, item):

  k_neighbors = k_neighbors_nearest(df, similarities, k, user, item)
  
  sum = 0
  sum_similarity = 0

  for neighbor_item in k_neighbors:
    if(neighbor_item != -1): #se nao deu o numero maximo de vizinhos mais proximos
      rating_neighbor_item = df.loc[user][neighbor_item]
    
      similarity = similarity_itens(similarities, neighbor_item, item)

      sum_similarity+= similarity
      
      sum+= similarity * rating_neighbor_item
  
  if sum_similarity ==0 and sum ==0:
    return 0
  elif sum_similarity ==0 and sum !=0:
    return sum
  else:
    return sum/sum_similarity

In [ ]:
items_mean = np.mean(df)
items_mean

In [ ]:
user_mean = np.mean(df.T)
user_mean

In [ ]:
total_mean = np.nanmean(df.values.tolist()) 
total_mean

3.603814223642363

In [ ]:
k = 3

l= []
for item,row in tqdm(df_test_data.iterrows()):
  user = row['user_id']
  item = row['movie_id']
  if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
    l.append(items_mean[item])
  elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l.append(user_mean[user])
  elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
    l.append(total_mean)
  else:
    l.append(pred(df, data, k, user, item))

df_test_data['rating'] = l
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FC_Items_' + str(k) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)